In [ ]:
import sys
import os

# Determine the notebook directory
notebook_dir = os.getcwd()

# Add the project directory to the sys.path
project_path = os.path.abspath(os.path.join(notebook_dir, '../..'))
if project_path not in sys.path:
    sys.path.append(project_path)

# Verify the PYTHONPATH
print(sys.path)

In [89]:
# Import BreezeAPI
from api.icicidirect.breeze_api import BreezeAPI

# Initialize BreezeAPI with credentials from config
api = BreezeAPI()

# Test: Fetch historical data for a specific stock
try:
    historical_data = api.get_historical_data(
        stock_code="NIFTY",
        interval="5minute",
        from_date="10/06/2000",  # Pass dates in DD/MM/YYYY format
        to_date="11/06/2013",     # Pass dates in DD/MM/YYYY format
    )
    
    # Print the retrieved historical data
    if historical_data:
        print("Historical Data:", historical_data)
    else:
        print("No data retrieved.")

except Exception as e:
    print(f"Error while fetching historical data: {e}")

Historical Data: {'Error': None, 'Status': 200, 'Success': [{'close': 5977.1, 'datetime': '2013-05-23 14:30:00', 'exchange_code': 'NSE', 'high': 5983.5, 'low': 5973.65, 'open': 5982.45, 'stock_code': 'NIFTY', 'volume': 0}, {'close': 5967.45, 'datetime': '2013-05-23 14:35:00', 'exchange_code': 'NSE', 'high': 5978.7, 'low': 5963.75, 'open': 5977.2, 'stock_code': 'NIFTY', 'volume': 0}, {'close': 5963.2, 'datetime': '2013-05-23 14:40:00', 'exchange_code': 'NSE', 'high': 5969.3, 'low': 5962.4, 'open': 5967.6, 'stock_code': 'NIFTY', 'volume': 0}, {'close': 5973.2, 'datetime': '2013-05-23 14:45:00', 'exchange_code': 'NSE', 'high': 5973.2, 'low': 5962.25, 'open': 5963.35, 'stock_code': 'NIFTY', 'volume': 0}, {'close': 5974.05, 'datetime': '2013-05-23 14:50:00', 'exchange_code': 'NSE', 'high': 5976.45, 'low': 5971.4, 'open': 5973.2, 'stock_code': 'NIFTY', 'volume': 0}, {'close': 5970.65, 'datetime': '2013-05-23 14:55:00', 'exchange_code': 'NSE', 'high': 5974.0, 'low': 5967.65, 'open': 5973.95, 

In [ ]:
(historical_data['Success'][0]['datetime'])

In [84]:
from breeze_connect import BreezeConnect
import pandas as pd
from datetime import datetime, timedelta
import time
#from configs.api_config import BREEZE_API_KEY, BREEZE_SECRET_KEY, BREEZE_SESSION_TOKEN


# Replace these with your actual Breeze API credentials
API_KEY = '9s(`2531H7371+7#48Jr#9MBR6649970'
API_SECRET = '79b372ny23+61@7Z0@m699~T56o9K07R'
SESSION_TOKEN = '48028761'

# Initialize BreezeConnect
api = BreezeConnect(api_key=API_KEY)
api.generate_session(api_secret=API_SECRET, session_token=SESSION_TOKEN)




In [85]:
from datetime import datetime

def convert_date(input_date):
    try:
        # Try to parse if the input is in "2018-12-13 11:05:00" format
        date_obj = datetime.strptime(input_date, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        # If the format doesn't match, assume it's in "dd/mm/yyyy" format
        date_obj = datetime.strptime(input_date, "%d/%m/%Y")
    
    # Return the formatted date in the required API format with time set to 07:00:00
    return date_obj.strftime("%Y-%m-%dT07:00:00.000Z")



In [ ]:
def fetch_and_store_historical_data(stock_code, interval, from_date, to_date,x):
    # Convert the input dates to the required format for the API
    def convert_date(input_date):
        try:
            date_obj = datetime.strptime(input_date, "%Y-%m-%d %H:%M:%S")
        except ValueError:
            date_obj = datetime.strptime(input_date, "%d/%m/%Y")
        return date_obj.strftime("%Y-%m-%dT07:00:00.000Z")

    # Convert `from_date` to datetime object for comparison
    from_date_obj = datetime.strptime(from_date, "%d/%m/%Y")

    # Initialize a DataFrame to store all the data
    all_data = pd.DataFrame()

    # Format the initial `to_date` in the required format
    current_to_date = convert_date(to_date)

    # Iterate until the earliest `datetime` in the response is earlier than `from_date`
    while True:
        # Fetch data from the Breeze API
        historical_data = api.get_historical_data_v2(
            interval=interval,
            from_date=convert_date(from_date),
            to_date=current_to_date,
            stock_code=stock_code,
            exchange_code="NSE",
            product_type="cash"
        )

        # Check if data was returned successfully
        if 'Success' in historical_data:
            data = historical_data['Success']
            df = pd.DataFrame(data)
            x = historical_data
            # Get the earliest datetime in the response as a string
            earliest_datetime_str = historical_data['Success'][0]['datetime']
            
            # Convert earliest_datetime_str to a datetime object for comparison
            earliest_datetime = datetime.strptime(earliest_datetime_str, "%Y-%m-%d %H:%M:%S")

            # Append the fetched data to the all_data DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)

            # If the earliest date in the response is earlier than `from_date`, stop the loop
            if earliest_datetime < from_date_obj:
                break

            # Otherwise, set the new `to_date` as the earliest datetime minus one second
            current_to_date = (earliest_datetime - timedelta(seconds=1)).strftime("%Y-%m-%dT07:00:00.000Z")
        else:
            print("Failed to fetch data.")
            break

    # Save the final DataFrame to a CSV file
    file_name = f"{stock_code}_{interval}.csv"
    all_data.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")

x = None
# Example usage
fetch_and_store_historical_data("NIFTY", "5minute", "01/08/2020", "07/08/2022",x)


In [86]:
# Convert the input dates to the required format for the API
stock_code = "NIFTY"
interval = "1minute"
from_date = "01/01/2000"
to_date = "09/10/2024"

def convert_date(input_date):
    try:
        date_obj = datetime.strptime(input_date, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        date_obj = datetime.strptime(input_date, "%d/%m/%Y")
    return date_obj.strftime("%Y-%m-%dT07:00:00.000Z")

# Convert `from_date` to datetime object for comparison
from_date_obj = datetime.strptime(from_date, "%d/%m/%Y")

# Initialize a DataFrame to store all the data
all_data = pd.DataFrame()

# Format the initial `to_date` in the required format
current_to_date = convert_date(to_date)

# Iterate until the earliest `datetime` in the response is earlier than `from_date`
while True:
    # Fetch data from the Breeze API
    historical_data = api.get_historical_data_v2(
        interval=interval,
        from_date=convert_date(from_date),
        to_date=current_to_date,
        stock_code=stock_code,
        exchange_code="NSE",
        product_type="cash"
    )

    # Check if data was returned successfully
    if 'Success' in historical_data:
        data = historical_data['Success']
        df = pd.DataFrame(data)
        x = historical_data
        # Get the earliest datetime in the response as a string
        earliest_datetime_str = historical_data['Success'][0]['datetime']
        
        # Convert earliest_datetime_str to a datetime object for comparison
        earliest_datetime = datetime.strptime(earliest_datetime_str, "%Y-%m-%d %H:%M:%S")

        # Append the fetched data to the all_data DataFrame
        all_data = pd.concat([all_data, df], ignore_index=True)

        # If the earliest date in the response is earlier than `from_date`, stop the loop
        if earliest_datetime < from_date_obj:
            break

        # Otherwise, set the new `to_date` as the earliest datetime minus one second
        current_to_date = (earliest_datetime - timedelta(seconds=1)).strftime("%Y-%m-%dT07:00:00.000Z")
    else:
        print("Failed to fetch data.")
        break

# Save the final DataFrame to a CSV file
file_name = f"{stock_code}_{interval}.csv"
all_data.to_csv(file_name, index=False)
print(f"Data saved to {file_name}")


Failed to fetch data.
Data saved to NIFTY_1minute.csv


In [ ]:
file_name = f"{stock_code}_{interval}.csv"
all_data.to_csv(file_name, index=False)
print(f"Data saved to {file_name}")

In [ ]:
#remove duplicate rows in the dataframe all_data and save it to a csv file
all_data = all_data.drop_duplicates()
all_data.to_csv(file_name, index=False)
print(f"Data saved to {file_name}")
